# Creating a guide starSolo Ref
    #Looks like the genome is needed 
    # This will make it imposible to align to guide sequences not present in the genome
    # It will try to map guides against other genomic regions creating a mix between real guides and genomic regions
    

# converting the metadata in a guide ID

In [145]:
!mkdir /home/jovyan/data/

In [146]:
import numpy as np
import pandas as pd
df = pd.read_csv("/mnt/shared/counting_guides/metadata/TF_CRISPR_i_a.scRNAseq_screen.igvf_metadata.txt", sep = '\t')

def get_index_str(c):
    if c.targeting:
        return f"{c['guide_chr']}_{int(c['guide_start'])}_{int(c['guide_end'])}_"\
        f"{c['strand']}_{c['protospacer']}_{c['intended_target_name']}"
    else:
        return f"{c['type']}_{c['protospacer']}"
        
df.index = [get_index_str(f[1]) for f in df.iterrows()]


with open("/home/jovyan/data/TF_CRISPR_i_a.scRNAseq_screen.protospaces.fa","w") as ofile:
    for l in zip(df.index, df.protospacer):
        ofile.write(">" + l[0] + "\n")
        ofile.write(l[1] + "\n")

# Creating a ref Lets use a guides as a chromossome

In [115]:
import pandas as pd
df_metadata_raw = pd.read_csv('/mnt/shared/counting_guides/metadata/TF_CRISPR_i_a.scRNAseq_screen.igvf_metadata.txt', sep='\t')

def revcomp(s):
    rev = dict(zip("ACTG", "TGAC"))
    return "".join([rev[c]for c in s[::-1]])


df_metadata_raw['protospacer'] =  'NNN'+ df_metadata_raw['protospacer'].apply(revcomp)


In [143]:
!cp '/mnt/shared/counting_guides/metadata/TF_CRISPR_i_a.scRNAseq_screen.igvf_metadata.txt' .

In [116]:
# df_metadata_raw.head()
# gff3_pd = pd.DataFrame()
# gff3_pd['seqid'] =  ("chr_peseudo_chr_" + df_metadata_raw['protospacer']).tolist()
# gff3_pd['source'] = '.'
# gff3_pd['type'] = 'mRNA'
# gff3_pd['star'] = 0


# gff3_pd['end'] = df_metadata_raw['protospacer'].apply(len).tolist()
# gff3_pd['score'] = 100
# gff3_pd['strand'] = '+'
# gff3_pd['phase'] = '.'
# gff3_pd['attribute'] = df_metadata_raw.apply(lambda x : f'ID={x["protospacer"]};Name={x["protospacer"]}', axis=1).tolist()
# gff3_pd.to_csv('guide_ref.gff', header=None, index=None, sep='\t')

In [117]:
df_metadata_raw.head()
bed_pd = pd.DataFrame()
bed_pd['seqid'] =  ("chr_peseudo_chr_" + df_metadata_raw['protospacer']).tolist()
bed_pd['star'] = 0
bed_pd['end'] = df_metadata_raw['protospacer'].apply(len) 
bed_pd['source'] =  df_metadata_raw['protospacer']
bed_pd['type'] = df_metadata_raw['protospacer']
bed_pd['strand'] = '+'
bed_pd['thickStart'] = 1
bed_pd['thickEnd'] = bed_pd['end'] 
bed_pd['color'] = "0,0,255"
bed_pd['blockCount'] = 1
bed_pd['blockSizes'] = bed_pd['end'] 
bed_pd['blockStarts'] = 0
bed_pd.to_csv('test.bed', header=None, index=None, sep='\t')


In [118]:
# > cat isoforms.txt

# ENSG00000198888 ENST00000361390
# ENSG00000198763 ENST00000361453
# ENSG00000198804 ENST00000361624
# ENSG00000188868 ENST00000595977

In [119]:
# name - Label to be displayed under the feature, if turned on in "Configure this page".
# score - A score between 0 and 1000. See track lines, below, for ways to configure the display style of scored data.
# strand - defined as + (forward) or - (reverse).
# thickStart - coordinate at which to start drawing the feature as a solid rectangle
# thickEnd - coordinate at which to stop drawing the feature as a solid rectangle
# itemRgb - an RGB colour value (e.g. 0,0,255). Only used if there is a track line with the value of itemRgb set to "on" (case-insensitive).
# blockCount - the number of sub-elements (e.g. exons) within the feature
# blockSizes - the size of these sub-elements
# blockStarts - the start coordinate of each sub-element

In [122]:
bed_pd[['seqid', 'source']].to_csv('isoforms.txt', header=None, index=None, sep='\t')




In [123]:
!bed2gtf --bed test.bed --output test.gtf --isoforms 'isoforms.txt'


##### BED2GTF #####
A fast BED-to-GTF converter written in Rust.
Repository: https://github.com/alejandrogzi/bed2gtf
Feel free to contact the developer if any issue/bug is found.
Version: 1.9.1

2024-02-23T20:44:43.869Z INFO  [bed2gtf] Using 7 threads
2024-02-23T20:44:43.872Z INFO  [bed2gtf] Memory usage: 0.2020626 MB
2024-02-23T20:44:43.872Z INFO  [bed2gtf] Elapsed: 0.0026 secs


In [124]:
# ##gff-version 3
# ctg123 . mRNA            1300  9000  .  +  .  ID=mrna0001;Name=sonichedgehog
# seqid - name of the chromosome or scaffold; chromosome names can be given with or without the 'chr' prefix. Important note: the seq ID must be one used within Ensembl, i.e. a standard chromosome name or an Ensembl identifier such as a scaffold ID, without any additional content such as species or assembly. See the example GFF output below.
# source - name of the program that generated this feature, or the data source (database or project name)
# type - type of feature. Must be a term or accession from the SOFA sequence ontology
# start - Start position of the feature, with sequence numbering starting at 1.
# end - End position of the feature, with sequence numbering starting at 1.
# score - A floating point value.
# strand - defined as + (forward) or - (reverse).
# phase - One of '0', '1' or '2'. '0' indicates that the first base of the feature is the first base of a codon, '1' that the second base is the first base of a codon, and so on..
# attributes - A semicolon-separated list of tag-value pairs, providing additional information about each feature. Some of these tags are predefined, e.g. ID, Name, Alias, Parent - see the GFF documentation for more details.


In [125]:
with open('pseudo_genome.fa', 'w') as save_genome:
    
    save_genome.write('\n'.join(df_metadata_raw.apply(lambda x : f'>chr_peseudo_chr_{x["protospacer"]}\n{x["protospacer"]}' ,axis=1 ).tolist()))

In [126]:
!STAR

Usage: STAR  [options]... --genomeDir /path/to/genome/index/   --readFilesIn R1.fq R2.fq
Spliced Transcripts Alignment to a Reference (c) Alexander Dobin, 2009-2022

STAR version=2.7.11b
STAR compilation time,server,dir=2024-01-29T15:15:38+0000 :/opt/conda/conda-bld/star_1706541070242/work/source
For more details see:
<https://github.com/alexdobin/STAR>
<https://github.com/alexdobin/STAR/blob/master/doc/STARmanual.pdf>

To list all parameters, run STAR --help


In [127]:
!mkdir new_ref

mkdir: cannot create directory ‘new_ref’: File exists


In [128]:
!STAR --runThreadN 2 --runMode genomeGenerate --genomeDir new_ref  --genomeFastaFiles pseudo_genome.fa --genomeSAindexNbases 3 --sjdbGTFfile test.gtf


	/opt/conda/bin/STAR-avx2 --runThreadN 2 --runMode genomeGenerate --genomeDir new_ref --genomeFastaFiles pseudo_genome.fa --genomeSAindexNbases 3 --sjdbGTFfile test.gtf
	STAR version: 2.7.11b   compiled: 2024-01-29T15:15:38+0000 :/opt/conda/conda-bld/star_1706541070242/work/source
Feb 23 20:44:48 ..... started STAR run
Feb 23 20:44:48 ... starting to generate Genome files
Feb 23 20:44:48 ..... processing annotations GTF
Feb 23 20:44:48 ... starting to sort Suffix Array. This may take a long time...
Feb 23 20:44:48 ... sorting Suffix Array chunks and saving them to disk...
Feb 23 20:44:48 ... loading chunks from disk, packing SA...
Feb 23 20:44:48 ... finished generating suffix array
Feb 23 20:44:48 ... generating Suffix Array index
Feb 23 20:44:48 ... completed Suffix Array index
Feb 23 20:44:48 ... writing Genome to disk ...
Feb 23 20:44:48 ... writing Suffix Array to disk ...
Feb 23 20:44:48 ... writing SAindex to disk
Feb 23 20:44:48 ..... finished successfully


# Downsampling

In [129]:

!zcat  /mnt/shared/counting_guides/fastq/gRNA_CRISPRi_D3_S9_L001_R1_001.fastq.gz  |  head -n 400000 >test_R1_001.fastq
!zcat /mnt/shared/counting_guides/fastq/gRNA_CRISPRi_D3_S9_L001_R2_001.fastq.gz | head -n 400000 > test_R2_001.fastq



gzip: stdout: Broken pipe

gzip: stdout: Broken pipe


In [141]:
cmd ='''
STAR --soloType CB_UMI_Simple \
--readFilesIn  test_R1_001.fastq  test_R2_001.fastq    \
--genomeDir /home/jovyan/new_ref  \
--soloCBwhitelist /mnt/shared/counting_guides/metadata/737K-august-2016.txt   \
--soloCBstart 1 \
--soloCBlen 16 \
--soloUMIstart 17 \
--soloUMIlen 10 \
--soloBarcodeMate 1 \
--soloCBmatchWLtype 1MM_multi_Nbase_pseudocounts \
--soloUMIfiltering MultiGeneUMI_CR \
--soloUMIdedup 1MM_CR \
--outFilterScoreMin 30 \
--outSAMtype BAM SortedByCoordinate \
--outSAMattributes CR UR CY UY CB UB \
--soloBarcodeReadLength 0 \
--clip5pNbases 39 0  \
--outFilterScoreMinOverLread 0  --outFilterMatchNminOverLread 0 \
#--readFilesCommand zcat \
'''

!$cmd

	/opt/conda/bin/STAR-avx2 --soloType CB_UMI_Simple --readFilesIn test_R1_001.fastq test_R2_001.fastq --genomeDir /home/jovyan/new_ref --soloCBwhitelist /mnt/shared/counting_guides/metadata/737K-august-2016.txt --soloCBstart 1 --soloCBlen 16 --soloUMIstart 17 --soloUMIlen 10 --soloBarcodeMate 1 --soloCBmatchWLtype 1MM_multi_Nbase_pseudocounts --soloUMIfiltering MultiGeneUMI_CR --soloUMIdedup 1MM_CR --outFilterScoreMin 30 --outSAMtype BAM SortedByCoordinate --outSAMattributes CR UR CY UY CB UB --soloBarcodeReadLength 0 --clip5pNbases 39 0 --outFilterScoreMinOverLread 0 --outFilterMatchNminOverLread 0
	STAR version: 2.7.11b   compiled: 2024-01-29T15:15:38+0000 :/opt/conda/conda-bld/star_1706541070242/work/source
Feb 23 20:58:38 ..... started STAR run
Feb 23 20:58:38 ..... loading genome
Feb 23 20:58:38 ..... started mapping
Feb 23 20:58:49 ..... finished mapping
Feb 23 20:58:49 ..... started Solo counting
Feb 23 20:58:50 ..... finished Solo counting
Feb 23 20:58:50 ..... started sorting B

In [ ]:
--soloType CB_UMI_Simple 
--soloCBstart 1 --soloCBlen 16 --soloUMIstart 17 --soloUMIlen 10 
--soloCBmatchWLtype 1MM_multi_Nbase_pseudocounts
--soloUMIfiltering MultiGeneUMI_CR
--soloBarcodeMate 1 --clip5pNbases 39 0 --soloUMIdedup 1MM_CR --outFilterScoreMin 30 --outSAMtype BAM SortedByCoordinate --outSAMattributes CR UR CY UY CB UB


In [133]:
import anndata as ann

ann_results = ann.read_mtx('Solo.out/Gene/raw/matrix.mtx')

ann_results

AnnData object with n_obs × n_vars = 40 × 737280

In [134]:
ann_results.X.sum(0)

matrix([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [27]:
!wget https://github.com/10XGenomics/cellranger/raw/master/lib/python/cellranger/barcodes/737K-august-2016.txt



--2024-02-23 19:13:03--  https://github.com/10XGenomics/cellranger/raw/master/lib/python/cellranger/barcodes/737K-august-2016.txt
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-02-23 19:13:03 ERROR 404: Not Found.



In [29]:
!ls 

737K-august-2016.txt
TF_CRISPR_i_a.scRNAseq_screen.igvf_metadata.txt
TF_CRISPR_i_a.scRNAseq_screen.seqspec.yaml


In [55]:
#!mamba install bed2gtf -c bioconda --y


Looking for: ['bed2gtf']

warning  libmamba Cache file "/opt/conda/pkgs/cache/ffeee55f.json" was modified by another program
bioconda/linux-64                                           Using cache
warning  libmamba Cache file "/opt/conda/pkgs/cache/2a957770.json" was modified by another program
bioconda/noarch                                             Using cache
warning  libmamba Cache file "/opt/conda/pkgs/cache/497deca9.json" was modified by another program
conda-forge/linux-64                                        Using cache
warning  libmamba Cache file "/opt/conda/pkgs/cache/09cdf8bf.json" was modified by another program
conda-forge/noarch                                          Using cache

Pinned packages:
  - python 3.11.*
  - python 3.11.6


Transaction

  Prefix: /opt/conda

  Updating specs:

   - bed2gtf
   - ca-certificates
   - certifi
   - openssl


  Package    Version  Build       Channel      Size
─────────────────────────────────────────────────────
  Install:
